# Pozisyonların Yeniden Dengelenmesi

**PyBroker** bir portföyün yeniden dengelenmesini simüle etmek için kullanılabilir. Bu, **PyBroker**'ın bir portföyün varlık tahsisini istenen hedef tahsise uyacak şekilde ayarlamayı simüle edebileceği anlamına gelir. Ek olarak, bu not defterinin de göstereceği gibi, portföyümüz [portföy optimizasyonu](https://en.wikipedia.org/wiki/Portfolio_optimization) kullanılarak yeniden dengelenebilir.

In [19]:
import pybroker as pyb
from datetime import datetime
from pybroker import ExecContext, Strategy, YFinance

pyb.enable_data_source_cache('rebalancing')

## Eşit Konum Boyutlandırması

Portföyümüzdeki her varlığın kabaca eşit bir tahsise sahip olduğundan emin olmak için her ayın başında sadece uzun vadeli bir portföyü yeniden dengelemek istediğimizi varsayalım.

Öncelikle mevcut çubuğun tarihinin yeni ayın başlangıcı olduğunu tespit edecek bir yardımcı fonksiyon yazarak başlıyoruz:

In [20]:
def start_of_month(dt: datetime) -> bool:
    if dt.month != pyb.param('current_month'):
        pyb.param('current_month', dt.month)
        return True
    return False

Daha sonra, hedef tahsise ulaşmak için bir varlıkta yeterli miktarda hisse satın alacak veya satacak bir fonksiyon uyguluyoruz.

In [21]:
def set_target_shares(
    ctxs: dict[str, ExecContext], 
    targets: dict[str, float]
):
    for symbol, target in targets.items():
        ctx = ctxs[symbol]
        target_shares = ctx.calc_target_shares(target)
        pos = ctx.long_pos()
        if pos is None:
            ctx.buy_shares = target_shares
        elif pos.shares < target_shares:
            ctx.buy_shares = target_shares - pos.shares
        elif pos.shares > target_shares:
            ctx.sell_shares = pos.shares - target_shares

Mevcut tahsis hedef seviyenin üzerindeyse fonksiyon varlığın bir kısmını satacak, mevcut tahsis hedef seviyenin altındaysa fonksiyon varlığın bir kısmını satın alacaktır.

Bunu takiben, her ayın başında her varlığı eşit hedef tahsisine ayarlamak için bir ``yeniden dengeleme`` işlevi yazıyoruz:

In [22]:
def rebalance(ctxs: dict[str, ExecContext]):
    dt = tuple(ctxs.values())[0].dt
    if start_of_month(dt):
        target = 1 / len(ctxs)
        set_target_shares(ctxs, {symbol: target for symbol in ctxs.keys()})

Now that we have implemented the ``rebalance`` function, the next step is to backtest our rebalancing strategy using five different stocks in our portfolio. To process all stocks at once on each bar of data, we will use the [Strategy#set_after_exec](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_after_exec) method:

In [23]:
strategy = Strategy(YFinance(), start_date='1/1/2018', end_date='1/1/2023')
strategy.add_execution(None, ['FROTO.IS', 'DOAS.IS', 'FROTO.IS', 'EREGL.IS', 'ISMEN.IS'])
strategy.set_after_exec(rebalance)
result = strategy.backtest()

Backtesting: 2018-01-01 00:00:00 to 2023-01-01 00:00:00

Loaded cached bar data.

Test split: 2018-01-01 00:00:00 to 2022-12-30 00:00:00


  0% (0 of 1274) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  5% (71 of 1274) |#                     | Elapsed Time: 0:00:00 ETA:  00:00:00
 13% (171 of 1274) |##                   | Elapsed Time: 0:00:00 ETA:   0:00:00
 19% (251 of 1274) |####                 | Elapsed Time: 0:00:00 ETA:   0:00:00
 25% (331 of 1274) |#####                | Elapsed Time: 0:00:00 ETA:   0:00:00
 33% (421 of 1274) |######               | Elapsed Time: 0:00:00 ETA:   0:00:00
 39% (501 of 1274) |########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 47% (601 of 1274) |#########            | Elapsed Time: 0:00:00 ETA:   0:00:00
 55% (701 of 1274) |###########          | Elapsed Time: 0:00:00 ETA:   0:00:00
 62% (791 of 1274) |#############        | Elapsed Time: 0:00:00 ETA:   0:00:00
 68% (871 of 1274) |##############       | Elapsed Time: 0:00:00 ETA:   0:00:00
 75% (961 of 1274) |###############      | Elapsed Time: 0:00:00 ETA:   0:00:00
 81% (1041 of 1274) |################   


Finished backtest: 0:00:01


In [24]:
result.orders

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,DOAS.IS,2018-01-02,3033,NaN,8.32,0.0
2,buy,FROTO.IS,2018-01-02,414,NaN,61.82,0.0
3,buy,ISMEN.IS,2018-01-02,43115,NaN,0.58,0.0
4,buy,EREGL.IS,2018-01-02,2373,NaN,10.18,0.0
5,sell,DOAS.IS,2018-02-02,138,NaN,8.98,0.0
...,...,...,...,...,...,...,...
236,buy,EREGL.IS,2022-11-02,1221,NaN,30.29,0.0
237,sell,ISMEN.IS,2022-12-02,670,NaN,11.01,0.0
238,sell,EREGL.IS,2022-12-02,740,NaN,41.88,0.0


## Portföy Optimizasyonu

[Portföy optimizasyonu](https://en.wikipedia.org/wiki/Portfolio_optimization), portföyümüz için bazı hedeflere ulaşmak amacıyla yeniden dengelememize rehberlik edebilir. Örneğin, varlıkları riski en aza indirecek şekilde tahsis etmek amacıyla portföy optimizasyonunu kullanabiliriz.

[Riskfolio-Lib](https://riskfolio-lib.readthedocs.io/), portföy optimizasyonu gerçekleştirmek için kullanılan popüler bir Python kitaplığıdır. Aşağıda, portföyün geçen yıla dayalı olarak [Koşullu Riske Maruz Değeri (CVar)](https://www.investopedia.com/terms/c/conditional_value_at_risk.asp) değerini en aza indirerek minimum risk portföyü oluşturmak için nasıl kullanılacağı gösterilmektedir. İadeler:

In [25]:
import pandas as pd
import riskfolio as rp

pyb.param('lookback', 252)  # Geçmiş yılın getirilerini kullanın.

def calculate_returns(ctxs: dict[str, ExecContext], lookback: int):
    prices = {}
    for ctx in ctxs.values():
        prices[ctx.symbol] = ctx.adj_close[-lookback:]
    df = pd.DataFrame(prices)
    return df.pct_change().dropna()

def optimization(ctxs: dict[str, ExecContext]):
    lookback = pyb.param('lookback')
    first_ctx = tuple(ctxs.values())[0]
    if start_of_month(first_ctx.dt):
        Y = calculate_returns(ctxs, lookback)
        port = rp.Portfolio(returns=Y)
        port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
        w = port.optimization(
            model='Classic', 
            rm='CVaR', 
            obj='MinRisk', 
            rf=0,      # Risksiz oran.
            l=0,       # Riskten kaçınma faktörü.
            hist=True  # Tarihsel senaryoları kullanın.
        )
        targets = {
            symbol: w.T[symbol].values[0]
            for symbol in ctxs.keys()
        }
        set_target_shares(ctxs, targets)

Resmi belgelerde [Riskfolio-Lib](https://riskfolio-lib.readthedocs.io/) kullanımına ilişkin daha fazla bilgi ve örnek bulabilirsiniz. Şimdi stratejiyi geriye doğru test etmeye geçelim!

In [26]:
strategy.set_after_exec(optimization)
result = strategy.backtest(warmup=pyb.param('lookback'))

Backtesting: 2018-01-01 00:00:00 to 2023-01-01 00:00:00

Loaded cached bar data.

Test split: 2018-01-01 00:00:00 to 2022-12-30 00:00:00


  0% (0 of 1274) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
 22% (291 of 1274) |####                 | Elapsed Time: 0:00:00 ETA:  00:00:00
 25% (331 of 1274) |#####                | Elapsed Time: 0:00:00 ETA:   0:00:00
 29% (371 of 1274) |######               | Elapsed Time: 0:00:00 ETA:   0:00:00
 33% (421 of 1274) |######               | Elapsed Time: 0:00:00 ETA:   0:00:00
 36% (461 of 1274) |#######              | Elapsed Time: 0:00:00 ETA:   0:00:00
 39% (501 of 1274) |########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 43% (551 of 1274) |#########            | Elapsed Time: 0:00:00 ETA:   0:00:00
 47% (601 of 1274) |#########            | Elapsed Time: 0:00:00 ETA:   0:00:00
 49% (631 of 1274) |##########           | Elapsed Time: 0:00:00 ETA:   0:00:00
 52% (671 of 1274) |###########          | Elapsed Time: 0:00:00 ETA:   0:00:00
 55% (711 of 1274) |###########          | Elapsed Time: 0:00:00 ETA:   0:00:00
 58% (751 of 1274) |############        


Finished backtest: 0:00:02


In [27]:
result.orders.head()

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,DOAS.IS,2019-01-02,5315,NaN,4.34,0.0
2,buy,FROTO.IS,2019-01-02,69,NaN,48.69,0.0
3,buy,ISMEN.IS,2019-01-02,112603,NaN,0.49,0.0
4,buy,EREGL.IS,2019-01-02,2553,NaN,7.15,0.0
5,sell,ISMEN.IS,2019-02-04,3422,NaN,0.59,0.0
